# （練習）RNA-seqカウントデータの読み込み・前処理

In [7]:
# pandasのインポート
import pandas as pd
import numpy as np

In [2]:
count_file = "input/counts.txt"
gene_id_product_file = "input/gene_id_product.tsv"

## 課題１ pd.read_tableまたはpd_read_csvを使って カウントデータを読み込む
データフレーム名は`df`とすること。

```
# Program:featureCounts v1.6.2; Command:"../tools/subread-1.6.2-Linux-x86_64/bin/featureCounts" "-p" "-T" "8" "-t" "exon" "-g" "gene_id" "-a" "../reference/s288c_e.gff" "-o" "../featurecount/counts.txt" "SRR453566.sorted.bam" "SRR453567.sorted.bam" "SRR453568.sorted.bam" "SRR453569.sorted.bam" "SRR453570.sorted.bam" "SRR453571.sorted.bam" 
Geneid  Chr     Start   End     Strand  Length  SRR453566.sorted.bam    SRR453567.sorted.bam    SRR453568.sorted.bam    SRR453569.sorted.bam    SRR453570.sorted.bam    SRR453571.sorted.bam
gene_0001       NC_001133.9     1807    2169    -       363     1       3       2       0       0       1
gene_0002       NC_001133.9     2480    2707    +       228     0       0       0       0       0       0
gene_0003       NC_001133.9     7235    9016    -       1782    0       0       0       0       0       0
gene_0004... 以下省略
```
1行目はfeatureCountsの実行条件が記載されているだけなので無視する。<br>
2行目は列タイトルを表すヘッダー行。<br>
3行目以降からがデータ行, 一番左の列が遺伝子idになっているのでこれをインデックスに用いる。

`pd.read_table()` メソッドの`skiprows`、`index_col`オプションを指定して読み込む。<br>
`header`オプションを使う方法, `pd.read_csv()`メソッドを使ってもよい。

In [ ]:
# (.....)に必要なオプションを指定して実行
df = pd.read_table(.....)

読み込めたら、データ件数などの確認を行う<br>
`df.head()`  `df.sum()` `df.shape` など

In [ ]:
# 列名とインデックスが正しく読み込まれているかを確認　head()を使う


In [ ]:
# データ件数を確認　shapeを使う


以後の解析のため、列名を変更しておきます。  
下記２セルを実行

In [8]:
# 列名を変更するための対応表
names = {'SRR453566.sorted.bam': 'batch_1',
         'SRR453567.sorted.bam': 'batch_2',
         'SRR453568.sorted.bam': 'batch_3',
         'SRR453569.sorted.bam': 'chemostat_1',
         'SRR453570.sorted.bam': 'chemostat_2',
         'SRR453571.sorted.bam': 'chemostat_3'}

In [9]:
# rename_axisをaxis=1を適用して使い、列名を変更する
df = df.rename_axis(mapper=names, axis=1)

In [ ]:
# 確認
df.head()

## 課題２ ミトコンドリア上の遺伝子のカウントデータを除く  
df.Chrの値が NC_001224.1 に一致するものがミトコンドリアに該当するので、boolean indexingを使って除く。

In [11]:
# df.Chrの値が NC_001224.1 に一致しないものを抽出してdfに代入する
df = df[df.Chr 　　　　　　]

In [ ]:
# 正しく処理ができればデータ件数は6394件になるはず
df.shape

## 課題３　pd.read_tableを使ってgene_id_productを読み込む
1列目をインデックスとして用いること。ヘッダー行がないので、 `names=["gene_id", "product"]`オプションを指定して読み込むこと。  
データフレーム名は `gene_products` とすること。

`gene_products = pd.read_table(.....)`


In [ ]:
# (.....)に必要なオプションを指定して実行
gene_products = pd.read_table(.....)

In [ ]:
# 正しく読み込めているか確認
gene_products.head()

## 課題４　gene_productsにdfを連結する  
インデックス列を使って接続するので `join()` を使うのが良い。 <br> 
データフレーム名はdf_with_productとすること。<br>

`df_with_product = gene_products.join(...`


In [ ]:
# (.....)に必要なオプションを指定して実行
df_with_product = gene_products.join(.....)

本来であれば rRNA　にマップされたリードは無視したいので除く必要がある。  
gene_products には mRNA のデータしか含まれていないので、rRNA などのデータはこの時点で除かれる。  
`pd.merge()`や`pd.concat()`を用いてもよい。

In [ ]:
# 確認
df_with_product.head()

カウントデータ部分のみを切り出し

In [19]:
df_count = df_with_product[["batch_1","batch_2","batch_3","chemostat_1","chemostat_2","chemostat_3"]]
# スライスで指定するなら　df_with_product.iloc[:, 6:12]

__ファイルの保存__<br>
カレントフォルダにoutputフォルダを作成しておく。<br>
`df_with_product`をoutputフォルダにcount_preprocessed.tsvとして保存する。<br>
`df_count`をoutputフォルダにcount_raw.tsvとして保存する。<br>
`df.to_csv(sep="\t")`を用いる。

In [20]:
# アノテーション付きカウントデータの保存
# (.....)に必要なオプションを入れて実行
df_with_product.to_csv(.....)

In [21]:
# raw カウントのデータを保存
# (.....)に必要なオプションを入れて実行
df_count.to_csv(.....)

In [ ]:
%%bash
# シェルコマンド head を使って確認
head output/count_raw.tsv

## 課題５　リード数による正規化（FPM/RPM）
RPM = reads per million <br>
カウントを100万リードあたりのカウント数に揃え, 正規化する。<br>
カウントデータをコピーした`df_tmp`を使う

In [24]:
#カウントデータを別のデータフレームとしてコピーしておく
df_tmp = df_count.copy()

In [ ]:
# 列ごとのリード数の合計
sum_count =
# カウント数を列ごとのリード数の合計で割り,100万をかける
df_tmp =　

In [29]:
# ただしく処理できていれば、下記のようになるはず。
# リード数の合計が100万に揃っていることを確認。
df_tmp.sum()

batch_1        1000000.0
batch_2        1000000.0
batch_3        1000000.0
chemostat_1    1000000.0
chemostat_2    1000000.0
chemostat_3    1000000.0
dtype: float64

上記の処理を関数化しておく。

In [ ]:
def normalize_per_million_reads(df):
    ....
    ....

In [31]:
# 関数の適用
df_count_fpm = normalize_per_million_reads(df_count)

In [32]:
# 確認
df_count_fpm.sum()

batch_1        1000000.0
batch_2        1000000.0
batch_3        1000000.0
chemostat_1    1000000.0
chemostat_2    1000000.0
chemostat_3    1000000.0
dtype: float64

In [33]:
# FPM正規化を行った結果を保存
df_count_fpm.to_csv("output/count_fpm.tsv", sep="\t")

## 課題６　遺伝子長による正規化 (RPKM/FPKM)
FPKM = fragments per kilobase of exon per million reads mapped<br>
上で求めたFPMをさらに遺伝子長で割って1000をかければ良い。<br>
以下に示す1~4のいずれかの方法を用いること。

In [34]:
# テスト用にデータフレームをコピーしておく
df_tmp = df_count_fpm.copy()

In [35]:
gene_length = df_with_product["Length"]

__1. for ループを使う方法__

データフレームをforループで回すと、列名が取得できるのでそれを利用する。

In [ ]:
for col_name in df_tmp:
    ....

In [ ]:
# 確認
df_tmp.head()

または`iteritems()`を用いて

In [ ]:
df_tmp = df_count_fpm.copy()
for col_name, col in df_tmp.iteritems():
    ....

In [ ]:
# 確認
df_tmp.head()

__2. データフレームを転置してから計算する方法__

データフレームの転置は`df.T`を使う。

In [ ]:
# テスト用にFPMをコピー
df_tmp = 
# df_tmpを転置してFPMを遺伝子長で割り, 1000をかける
df_tmp = 
# 戻す（もう一度転置する）
df_tmp = df_tmp.T

In [ ]:
# 確認
df_tmp.head()

__3. applyを使い各列に関数を適用する方法__

In [41]:
# はじめに列を入力とし、各要素を遺伝子長でわる処理を行う関数を定義する
def divide_by_length(S):
    ...

In [ ]:
# テスト用にFPMをコピー
df_tmp = 
# 関数を適用する
df_tmp.apply(divide_by_length).head()

__4. データフレームメソッドのdivideを使用する方法__

In [ ]:
# データフレームメソッドのdivideを使用する方法
df_tmp = df_count_fpm.copy()
df_tmp = 

In [9]:
# 確認
df_tmp.head()

1~4のいずれかの方法を関数化して計算する

In [44]:
def normalize_per_kilobase(df, gene_length):
    ...

In [45]:
df_count_fpkm = normalize_per_kilobase(df_count_fpm, gene_length)

In [ ]:
# 確認
df_count_fpkm.head()

In [60]:
# 保存する
df_count_fpkm.to_csv("output/count_fpkm.tsv", sep="\t")

## TPM 正規化
TPM = transcripts per kilobase million  
TPM の説明については以下のページが詳しい https://bi.biopapyrus.jp/  
FPKM/RPKM のときとは逆に、長さ1,000bpあたりのリード数を求めてから、総リード数を100万に揃えれば良い。

In [61]:
df_tmp = df_count.copy()

In [62]:
df_tmp = normalize_per_kilobase(df_tmp, gene_length)
df_tmp = normalize_per_million_reads(df_tmp)


In [ ]:
df_tmp.head()

In [64]:
# RPKM/FPKMと違い、合計が100万となっています。
df_tmp.sum()

batch_1        1000000.0
batch_2        1000000.0
batch_3        1000000.0
chemostat_1    1000000.0
chemostat_2    1000000.0
chemostat_3    1000000.0
dtype: float64

In [65]:
# 関数化します
def normalize_tpm(df, gene_length):
    df_tmp = df.copy()
    df_tmp = normalize_per_kilobase(df_tmp, gene_length)
    df_tmp = normalize_per_million_reads(df_tmp)
    return df_tmp

In [66]:
df_count_tpm = normalize_tpm(df_count, gene_length)

In [67]:
df_count_tpm.sum() #  確認

batch_1        1000000.0
batch_2        1000000.0
batch_3        1000000.0
chemostat_1    1000000.0
chemostat_2    1000000.0
chemostat_3    1000000.0
dtype: float64

In [68]:
# 保存
df_count_tpm.to_csv("output/count_tpm.tsv", sep="\t")

## 課題7　発現変動遺伝子を抽出する
以下のセルを実行し, 各条件の平均から発現変動をlog2 foldとして求める。

In [74]:
# batch cultureの平均を求める
df_count_fpkm["batch"] = (df_count_fpkm["batch_1"] + df_count_fpkm["batch_2"] + df_count_fpkm["batch_3"]) / 3

# chemostatの平均を求める
df_count_fpkm["chemostat"] = (df_count_fpkm["chemostat_1"] + df_count_fpkm["chemostat_2"] + df_count_fpkm["chemostat_3"]) / 3

# 発現変動をlog2 fold として求める
# 0 での除算を防ぐため、分母に微小な値を加えている
df_count_fpkm["log2fold"] = df_count_fpkm["chemostat"] / (df_count_fpkm["batch"] + 10**-6)
df_count_fpkm["log2fold"] = df_count_fpkm["log2fold"].apply(np.log2)

In [ ]:
# 確認
df_count_fpkm.head()

`df_count_fpkm` から"batch", "chemostat", "log2fold"の列を抜き出し`diff_ex`とする

In [ ]:
# [.......]に抜き出す列のリストを入れて実行
diff_ex = df_count_fpkm[.......]

`diff_ex.join()` でproductと結合, productとgene_idの対応は`gene_products`を使う

In [78]:
# join()でproductと結合
diff_ex = 

In [ ]:
# 確認
diff_ex.head()

In [11]:
# カウント数が0であるデータを除いておく
diff_ex = diff_ex[diff_ex["batch"] > 0]
diff_ex = diff_ex[diff_ex["chemostat"] > 0]

In [10]:
# 確認
diff_ex.head()

`log2fold`の列を降順に並び替える（`diff_ex.sort_values()`を使う)

In [ ]:
# (.....)に必要なオプションを入れて実行
diff_ex = diff_ex.sort_values(.....)

In [ ]:
# chemostat > batch の上位5番目まで表示


In [ ]:
# batch > chemostat の上位5番目まで表示
